In [1]:
import tensorflow as tf

train_root = "data/train/"

test_root = "data/validation/"

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

image = (150, 150); split = 0.1; random = 32

unseet = 10; shuffle = True; label = "binary"

train = image_dataset_from_directory(
  train_root,
  seed = unseet,
  image_size = image,
  label_mode = label,
  shuffle = shuffle,
  batch_size = random,
)

test = image_dataset_from_directory(
  test_root,
  seed = unseet,
  image_size = image,
  label_mode = label,
  shuffle = shuffle,
  batch_size = random,
)

In [3]:
weights = 'model/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(include_top=False, input_shape=(150, 150, 3), weights=weights)

pre_trained_model.trainable = False

In [5]:
from tensorflow.keras.callbacks import Callback

class EarlyStoppingCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get("accuracy") > 0.999:
      self.model.stop_training = True

callbacks = [EarlyStoppingCallback()]

In [6]:
last_desired_layer = pre_trained_model.get_layer('mixed7')

last_output = last_desired_layer.output

In [7]:
x = tf.keras.layers.Flatten()(last_output)

x = tf.keras.layers.Dense(1024, activation="relu")(x)

x = tf.keras.layers.Dropout(0.2)(x)

output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=pre_trained_model.input, outputs=output)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001)

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

In [8]:
val_batches = int(test.cardinality())

test, validation = tf.keras.utils.split_dataset(test, val_batches//5)

In [9]:
def preprocess(image, label):
  """
  Helper Transfer Learning
  """
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  return image, label

train = train.map(preprocess)

validation = validation.map(preprocess)

test = test.map(preprocess)

In [ ]:
history = model.fit(train, validation_data = validation, epochs = 10, callbacks = callbacks)

In [11]:
model.evaluate(train, verbose=False)

[0.003932130988687277, 1.0]

In [12]:
model.evaluate(test, verbose=False)

[0.008428575471043587, 1.0]

In [13]:
model.evaluate(validation, verbose=False)

[0.03289664909243584, 0.9955357313156128]

In [14]:
total_loop = len(history.history["loss"])

total_loop

3